<a href="https://colab.research.google.com/github/Song-Jun-Ho/2NN/blob/main/2nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
#%cd 'COPY&PASTE FILE DIRECTORY HERE'
%cd /content/drive/MyDrive/HW3/HW3

/content/drive/MyDrive/HW3/HW3


# Import Modules

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [ ]:
def sigmoid(z):
    """
    Do NOT modify this function
    """
    return 1/(1+np.exp(-z))

def softmax(X):
    """
    Do NOT modify this function
    """
    logit = np.exp(X-np.amax(X, axis=1, keepdims=True))
    numer = logit
    denom = np.sum(logit, axis=1, keepdims=True)
    return numer/denom

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

def relu(X):
    mask = (X <= 0)
    out = X.copy()
    out[mask] = 0

    return out, mask

#2-Layer Neural Network

In [ ]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        scale = np.sqrt(1.0 / input_dim)
        params["W1"] = scale * np.random.randn(input_dim, num_hiddens)
        params["b1"] = np.zeros(num_hiddens)
        scale = np.sqrt(1.0 / num_hiddens)
        params["W2"] = scale * np.random.randn(num_hiddens, num_classes)
        params["b2"] = np.zeros(num_classes)

        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(sigmoid(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        ff_dict['Affine1_in'] = X
        X = np.dot(X, self.params['W1']) + self.params['b1']
        X = sigmoid(X)
        ff_dict['sigmoid_out'] = X
        ff_dict['Affine2_in'] = X
        X = np.dot(X, self.params['W2']) + self.params['b2']
        out = softmax(X)
        ff_dict['softmax_out'] = out
        
        return out, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        batch_size = X.shape[0]
        dx = (ff_dict['softmax_out'] - Y) / batch_size

        grads['dW2'] = np.dot(ff_dict['Affine2_in'].T, dx)
        grads['db2'] = np.sum(dx, axis=0)
        dx = np.dot(dx, self.params['W2'].T)

        dx = dx * (1.0 - ff_dict['sigmoid_out']) * ff_dict['sigmoid_out']

        grads['dW1'] = np.dot(ff_dict['Affine1_in'].T, dx)
        grads['db1'] = np.sum(dx, axis=0)
        dx = np.dot(dx, self.params['W1'].T)

        return grads

    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

#Load MNIST

In [ ]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [ ]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=64, num_classes=10)

In [ ]:
# train the model
lr, n_epochs, batch_size = 0.2, 1000, 100
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 1.648 0.602, valid loss/acc: 1.643 0.602
epoch 01 - train loss/acc: 1.192 0.728, valid loss/acc: 1.187 0.730
epoch 02 - train loss/acc: 0.935 0.785, valid loss/acc: 0.931 0.786
epoch 03 - train loss/acc: 0.786 0.816, valid loss/acc: 0.784 0.814
epoch 04 - train loss/acc: 0.691 0.836, valid loss/acc: 0.690 0.832
epoch 05 - train loss/acc: 0.626 0.848, valid loss/acc: 0.624 0.845
epoch 06 - train loss/acc: 0.577 0.857, valid loss/acc: 0.577 0.855
epoch 07 - train loss/acc: 0.540 0.866, valid loss/acc: 0.540 0.865
epoch 08 - train loss/acc: 0.511 0.871, valid loss/acc: 0.511 0.869
epoch 09 - train loss/acc: 0.488 0.875, valid loss/acc: 0.487 0.873
epoch 10 - train loss/acc: 0.468 0.880, valid loss/acc: 0.468 0.877
epoch 11 - train loss/acc: 0.451 0.883, valid loss/acc: 0.451 0.880
epoch 12 - train loss/acc: 0.437 0.886, valid loss/acc: 0.437 0.883
epoch 13 - train loss/acc: 0.425 0.888, valid loss/acc: 0.425 0.885
epoch 14 - train loss/acc: 0.415 0.891, valid lo

In [ ]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.093, acc = 0.972


In [ ]:
def initialize_parameters(self, input_dim, num_hiddens, num_classes):
    """
    initializes parameters with He Initialization.

    Inputs
    - input_dim
    - num_hiddens
    - num_classes
    Returns
    - params: a dictionary with the initialized parameters.
    """
    params = {}
    scale = np.sqrt(2.0 / input_dim)
    params["W1"] = scale * np.random.randn(input_dim, num_hiddens)
    params["b1"] = np.zeros(num_hiddens)
    scale = np.sqrt(2.0 / num_hiddens)
    params["W2"] = scale * np.random.randn(num_hiddens, num_classes)
    params["b2"] = np.zeros(num_classes)
    
    return params

def forward_relu(self, X):
    """
    Defines and performs the feed forward step of a two-layer neural network.
    Specifically, the network structue is given by

        y = softmax(relu(X W1 + b1) W2 + b2)

    where X is the input matrix of shape (N, D), y is the class distribution matrix
    of shape (N, C), N is the number of examples (either the entire dataset or
    a mini-batch), D is the feature dimensionality, and C is the number of classes.

    Question (e)

    Inputs
        X: the input matrix of shape (N, D)

    Returns
        y: the output of the model
        ff_dict: a dictionary containing all the fully connected units and activations.
    """
    ff_dict = {}
    ff_dict['Affine1_in'] = X
    X = np.dot(X, self.params['W1']) + self.params['b1']
    X, mask = relu(X)
    ff_dict['relu_mask'] = mask
    ff_dict['Affine2_in'] = X
    X = np.dot(X, self.params['W2']) + self.params['b2']
    out = softmax(X)
    ff_dict['softmax_out'] = out        
    
    return out, ff_dict

def backward_relu(self, X, Y, ff_dict):
    """
    Performs backpropagation over the two-layer neural network, and returns
    a dictionary of gradients of all model parameters.

    Inputs:
        - X: the input matrix of shape (B, D), where B is the number of examples
            in a mini-batch, D is the feature dimensionality.
        - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
            where B is the number of examples in a mini-batch, C is the number
            of classes.
        - ff_dict: the dictionary containing all the fully connected units and
            activations.

    Returns:
        - grads: a dictionary containing the gradients of corresponding weights
            and biases.
    """
    grads = {}
    batch_size = X.shape[0]
    dx = (ff_dict['softmax_out'] - Y) / batch_size

    grads['dW2'] = np.dot(ff_dict['Affine2_in'].T, dx)
    grads['db2'] = np.sum(dx, axis=0)
    dx = np.dot(dx, self.params['W2'].T)

    dx[ff_dict['relu_mask']] = 0

    grads['dW1'] = np.dot(ff_dict['Affine1_in'].T, dx)
    grads['db1'] = np.sum(dx, axis=0)
    dx = np.dot(dx, self.params['W1'].T)

    return grads

TwoLayerNNRelu = copy.copy(TwoLayerNN)
TwoLayerNNRelu.initialize_parameters = initialize_parameters
TwoLayerNNRelu.forward = forward_relu
TwoLayerNNRelu.backward = backward_relu

In [ ]:
# model instantiation
model_relu = TwoLayerNNRelu(input_dim=784, num_hiddens=64, num_classes=10)

In [ ]:
# train the model
lr, n_epochs, batch_size = 0.2, 1000, 100
history = model_relu.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 1.603 0.647, valid loss/acc: 1.602 0.645
epoch 01 - train loss/acc: 1.152 0.759, valid loss/acc: 1.151 0.759
epoch 02 - train loss/acc: 0.912 0.799, valid loss/acc: 0.911 0.798
epoch 03 - train loss/acc: 0.772 0.821, valid loss/acc: 0.772 0.818
epoch 04 - train loss/acc: 0.683 0.837, valid loss/acc: 0.683 0.833
epoch 05 - train loss/acc: 0.621 0.847, valid loss/acc: 0.622 0.844
epoch 06 - train loss/acc: 0.575 0.856, valid loss/acc: 0.576 0.854
epoch 07 - train loss/acc: 0.541 0.863, valid loss/acc: 0.542 0.859
epoch 08 - train loss/acc: 0.513 0.869, valid loss/acc: 0.514 0.865
epoch 09 - train loss/acc: 0.490 0.874, valid loss/acc: 0.491 0.870
epoch 10 - train loss/acc: 0.471 0.876, valid loss/acc: 0.473 0.874
epoch 11 - train loss/acc: 0.455 0.880, valid loss/acc: 0.457 0.875
epoch 12 - train loss/acc: 0.441 0.883, valid loss/acc: 0.443 0.879
epoch 13 - train loss/acc: 0.429 0.886, valid loss/acc: 0.431 0.881
epoch 14 - train loss/acc: 0.419 0.888, valid lo

In [ ]:
Y_hat, _ = model_relu.forward(X_test)
test_loss = model_relu.compute_loss(Y_test, Y_hat)
test_acc = model_relu.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.089, acc = 0.973
